In [6]:
# Import Dependencies
import pandas as pd
import json
import requests
from datetime import date, timedelta, datetime
#from config import api_key
from pymongo import MongoClient


In [2]:
# Save config information
url = "https://api.openweathermap.org/data/3.0/onecall/day_summary?"
lat = 41.881832
lon = -87.623177
units = "imperial"

In [3]:
#Define start and end dates, create a date list and a loop to add all desired dates to the date list
start_date = date(2022, 1, 1) 
end_date = date(2022, 12, 31)
current_date = start_date
date_list = []

while current_date <= end_date:
    date_list.append(current_date.strftime('%Y-%m-%d'))
    current_date+=timedelta(days=1)

In [4]:
#Create a for loop to retrieve OpenWeather data for all dates in the date list

#Set up response info lists
cloud_cover = []
precipitation = []
min_temp = []
max_temp = []
temp_mrng = []
temp_aft = []
temp_eve = []
temp_night = []
wind_max = []

# Loop through all the cities in our list to fetch weather data
for day in date_list:
        
# Create endpoint URL with each day
    query_url = f"{url}&lat={lat}&lon={lon}&units={units}&date={day}&appid={api_key}&q="

    response = requests.get(query_url).json()
    cloud_cover.append(response['cloud_cover']['afternoon'])
    precipitation.append(response['precipitation']['total'])
    min_temp.append(response['temperature']['min'])
    max_temp.append(response['temperature']['max'])
    temp_mrng.append(response['temperature']['morning'])
    temp_aft.append(response['temperature']['afternoon'])
    temp_eve.append(response['temperature']['evening'])
    temp_night.append(response['temperature']['night'])
    wind_max.append(response['wind']['max']['speed'])


In [5]:
weather_dict = {
    "date": date_list,
    "cloud_cover": cloud_cover,
    "precipitation": precipitation,
    "min_temp": min_temp,
    "max_temp": max_temp,
    "morning_temp": temp_mrng,
    "afternoon_temp": temp_aft,
    "evening_temp": temp_eve,
    "night_temp": temp_night,
    "max_windspeed": wind_max
}

weather_df = pd.DataFrame(weather_dict)
weather_df.head()

,date,cloud_cover,precipitation,min_temp,max_temp,morning_temp,afternoon_temp,evening_temp,night_temp,max_windspeed
0,2022-01-01,90.0,0.18,33.22,42.10,42.10,38.43,35.24,38.44,15.01
1,2022-01-02,90.0,13.81,18.81,32.77,28.24,26.64,19.63,32.77,18.41
2,2022-01-03,16.0,0.00,6.58,26.37,21.76,12.25,7.21,24.21,8.99
3,2022-01-04,4.0,0.00,18.03,30.36,18.03,20.97,21.29,19.26,10.00
4,2022-01-05,64.0,0.25,13.57,33.39,31.53,32.74,22.19,31.64,17.00


In [ ]:
#Export our weather dataframe to a csv file
weather_df.to_csv("../Resources/weather_daily.csv", index=False, header=True)

In [28]:
csv_file_path = 'weather_daily.csv'
weather_df = pd.read_csv(csv_file_path)

In [30]:
mongo = MongoClient(port=27017)
db = mongo.chicago_bikes
divvy_rides = db['divvy_ridedata']  

weather_records = weather_df.to_dict(orient='records')
weather_records
# Insert weather records into the collection
collection.insert_many(weather_records)

#print(collection.find_one())

# Close the MongoDB connection
#client.close()

[{'date': '2022-01-01',
  'cloud_cover': 90.0,
  'precipitation': 0.18,
  'min_temp': 273.83,
  'max_temp': 278.76,
  'morning_temp': 278.76,
  'afternoon_temp': 276.72,
  'evening_temp': 274.95,
  'night_temp': 276.73,
  'max_windspeed': 6.71},
 {'date': '2022-01-02',
  'cloud_cover': 90.0,
  'precipitation': 13.81,
  'min_temp': 265.82,
  'max_temp': 273.58,
  'morning_temp': 271.06,
  'afternoon_temp': 270.17,
  'evening_temp': 266.28,
  'night_temp': 273.58,
  'max_windspeed': 8.23},
 {'date': '2022-01-03',
  'cloud_cover': 16.0,
  'precipitation': 0.0,
  'min_temp': 259.03,
  'max_temp': 270.02,
  'morning_temp': 267.46,
  'afternoon_temp': 262.18,
  'evening_temp': 259.38,
  'night_temp': 268.82,
  'max_windspeed': 4.02},
 {'date': '2022-01-04',
  'cloud_cover': 4.0,
  'precipitation': 0.0,
  'min_temp': 265.39,
  'max_temp': 272.24,
  'morning_temp': 265.39,
  'afternoon_temp': 267.02,
  'evening_temp': 267.2,
  'night_temp': 266.07,
  'max_windspeed': 4.47},
 {'date': '2022-01-